<a href="https://colab.research.google.com/github/akashFG/Algorithmic-Trading-and-Samsung-Advisor/blob/main/Task1_Algorithmic_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Required Library**

In [ ]:
!pip install yfinance

**Import Libraries**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

**Class-Based Trading Strategy**

In [ ]:
class TradingStrategy:
    def __init__(self, symbol, start_date, end_date, capital=5000):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.capital = capital
        self.cash = capital
        self.position = 0
        self.buy_price = 0
        self.trades = []

    def fetch_data(self):
        df = yf.download(self.symbol, start=self.start_date, end=self.end_date)
        df = df[~df.index.duplicated()]
        df.fillna(method="ffill", inplace=True)
        self.data = df

    def compute_indicators(self):
        self.data["MA50"] = self.data["Close"].rolling(50).mean()
        self.data["MA200"] = self.data["Close"].rolling(200).mean()

    def run_strategy(self):
        for i in range(1, len(self.data)):
            today = self.data.iloc[i]
            yesterday = self.data.iloc[i - 1]

            # BUY (Golden Cross)
            if (yesterday["MA50"] < yesterday["MA200"] and
                today["MA50"] > today["MA200"] and
                self.position == 0):

                qty = int(self.cash / today["Close"])
                if qty > 0:
                    self.position = qty
                    self.buy_price = today["Close"]
                    self.cash -= qty * today["Close"]
                    self.trades.append(("BUY", today.name, today["Close"], qty))

            # SELL (Death Cross)
            elif (yesterday["MA50"] > yesterday["MA200"] and
                  today["MA50"] < today["MA200"] and
                  self.position > 0):

                self.cash += self.position * today["Close"]
                self.trades.append(("SELL", today.name, today["Close"], self.position))
                self.position = 0

        # Force close on last day
        if self.position > 0:
            last_price = self.data.iloc[-1]["Close"]
            self.cash += self.position * last_price
            self.trades.append(("FORCE SELL", self.data.index[-1], last_price, self.position))
            self.position = 0

    def evaluate(self):
        profit = self.cash - self.capital
        return profit, self.trades

**Run Strategy**

In [ ]:
def fetch_data(self):
    df = yf.download(
        self.symbol,
        start=self.start_date,
        end=self.end_date,
        auto_adjust=False
    )

    df = df[~df.index.duplicated()]
    df = df.ffill()
    self.data = df

In [ ]:
def run_strategy(self):
    for i in range(1, len(self.data)):
        ma50_today = self.data["MA50"].iloc[i]
        ma200_today = self.data["MA200"].iloc[i]
        ma50_yesterday = self.data["MA50"].iloc[i - 1]
        ma200_yesterday = self.data["MA200"].iloc[i - 1]
        close_price = self.data["Close"].iloc[i]

        # BUY (Golden Cross)
        if (
            ma50_yesterday < ma200_yesterday and
            ma50_today > ma200_today and
            self.position == 0
        ):
            qty = int(self.cash / close_price)
            if qty > 0:
                self.position = qty
                self.buy_price = close_price
                self.cash -= qty * close_price
                self.trades.append(("BUY", self.data.index[i], close_price, qty))

        # SELL (Death Cross)
        elif (
            ma50_yesterday > ma200_yesterday and
            ma50_today < ma200_today and
            self.position > 0
        ):
            self.cash += self.position * close_price
            self.trades.append(("SELL", self.data.index[i], close_price, self.position))
            self.position = 0

    # Force close on last day
    if self.position > 0:
        last_price = self.data["Close"].iloc[-1]
        self.cash += self.position * last_price
        self.trades.append(
            ("FORCE SELL", self.data.index[-1], last_price, self.position)
        )
        self.position = 0

In [ ]:
import types

strategy = TradingStrategy("AAPL", "2018-01-01", "2023-12-31")

# Dynamically bind the updated fetch_data function as a method to the strategy instance
strategy.fetch_data = types.MethodType(fetch_data, strategy)
strategy.fetch_data()

strategy.compute_indicators()

# Dynamically bind the updated run_strategy function as a method to the strategy instance
strategy.run_strategy = types.MethodType(run_strategy, strategy)
strategy.run_strategy()

profit, trades = strategy.evaluate()

for t in trades:
    print(t)

# Extract the scalar value from the profit Series before formatting
print(f"\nFinal Profit/Loss: ${profit.iloc[0]:.2f}")

[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4046003346.py:15: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  qty = int(self.cash / close_price)


('BUY', Timestamp('2019-05-07 00:00:00'), Ticker
AAPL    50.715
Name: 2019-05-07 00:00:00, dtype: float64, 98)
('SELL', Timestamp('2022-06-03 00:00:00'), Ticker
AAPL    145.380005
Name: 2022-06-03 00:00:00, dtype: float64, 98)
('BUY', Timestamp('2022-09-28 00:00:00'), Ticker
AAPL    149.839996
Name: 2022-09-28 00:00:00, dtype: float64, 95)
('SELL', Timestamp('2022-09-30 00:00:00'), Ticker
AAPL    138.199997
Name: 2022-09-30 00:00:00, dtype: float64, 95)
('BUY', Timestamp('2023-03-23 00:00:00'), Ticker
AAPL    158.929993
Name: 2023-03-23 00:00:00, dtype: float64, 82)
('FORCE SELL', Timestamp('2023-12-29 00:00:00'), Ticker
AAPL    192.529999
Name: 2023-12-29 00:00:00, dtype: float64, 82)

Final Profit/Loss: $10926.57
